In [156]:
import numpy as np

In [134]:
import os
import pandas as pd
import pyshorteners
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup

def load_data(path_excel, path_csv, sheet_name_value=2):
    data_excel = pd.read_excel(path_excel, 
                               sheet_name=sheet_name_value, 
                               skiprows=range(1, 6))
    data_excel['STATUS'] = data_excel['STATUS'].str.strip()
    data_excel = data_excel.drop_duplicates(subset='ITEM_ID', keep='first')
    data_excel = data_excel.drop(columns=['VARIATIONS', 'VARIATION_ID'])
    
    data_csv = path_csv

    return data_excel, data_csv

In [136]:
excel_path = 'Anuncios-2025_01_30-18_16.xlsx'
csv_1 = pd.read_csv('produtos_2025-01-30-19-13-38.csv')
csv_2 = pd.read_csv("produtos_2025-01-30-19-14-55.csv")

csv_df = pd.concat([csv_1, csv_2], ignore_index=True)
print(csv_df.head(3))

          ID     Código (SKU)  \
0  734436383  TH001-2412-0542   
1  734141570    005-2410-0011   
2  734142200    005-2410-0027   

                                           Descrição  Unidade  \
0           2k Essentials Collection - Ps3 - Lacrado      NaN   
1  Amiibo The Legend Of Zelda - Tears Of The King...      NaN   
2  Amiibo The Legend Of Zelda Breath Of The Wild ...      NaN   

   Classificação fiscal                                             Origem  \
0                   NaN  0 - Nacional, exceto as indicadas nos códigos ...   
1                   NaN                                                NaN   
2                   NaN                                                NaN   

    Preço Valor IPI fixo  Observações Situação  ... Dias para preparação  \
0    0,00           0,00          NaN    Ativo  ...                    0   
1  229,00           0,00          NaN    Ativo  ...                    0   
2  270,00           0,00          NaN    Ativo  ...              

In [138]:
excel_df, csv_df = load_data(excel_path, csv_df )
print(excel_df.columns)
print(csv_df.columns)

Index(['ITEM_ID', 'SKU', 'TITLE', 'QUANTITY', 'CHANNEL', 'MARKETPLACE_PRICE',
       'MSHOPS_PRICE', 'MSHOPS_PRICE_SYNC', 'CURRENCY_ID', 'DESCRIPTION',
       'SHIPPING_METHOD_MARKETPLACE', 'SHIPPING_METHOD_MSHOPS', 'LISTING_TYPE',
       'FEE_PER_SALE_MARKETPLACE', 'FEE_PER_SALE_MSHOPS', 'STATUS',
       'CATEGORY'],
      dtype='object')
Index(['ID', 'Código (SKU)', 'Descrição', 'Unidade', 'Classificação fiscal',
       'Origem', 'Preço', 'Valor IPI fixo', 'Observações', 'Situação',
       'Estoque', 'Preço de custo', 'Cód do Fornecedor', 'Fornecedor',
       'Localização', 'Estoque máximo', 'Estoque mínimo', 'Peso líquido (Kg)',
       'Peso bruto (Kg)', 'GTIN/EAN', 'GTIN/EAN tributável',
       'Descrição complementar', 'CEST', 'Código de Enquadramento IPI',
       'Formato embalagem', 'Largura embalagem', 'Altura embalagem',
       'Comprimento embalagem', 'Diâmetro embalagem', 'Tipo do produto',
       'URL imagem 1', 'URL imagem 2', 'URL imagem 3', 'URL imagem 4',
       'URL im

In [140]:
# excel_df = excel_df.drop('SKU', axis=1)
print(excel_df.shape)
csv_df = csv_df.rename(columns={'Descrição': 'TITLE','Código (SKU)':"SKU"})
print(csv_df.shape)

(787, 17)
(812, 64)


In [142]:
def fill_sku_(df_with_sku, df_without_sku):
    """Preenche os SKUs em df_without_sku com base em df_with_sku (usando ITEM_ID como chave)."""
    merged_data = df_without_sku.merge(df_with_sku[['TITLE', 'SKU']], on='TITLE', how='left', suffixes=('', '_from_df1'))
    merged_data['SKU'] = merged_data['SKU_from_df1']
    merged_data.drop(columns=['SKU_from_df1'], inplace=True)
    return merged_data

df = fill_sku_(csv_df, excel_df)

print(f"\n Final df \n{df.shape}")


 Final df 
(823, 17)


In [144]:
df['CATEGORY'].unique()

array(['Relógios de bolso', 'Gamepads e joysticks', 'Videogames',
       'Consoles de videogames', 'Produtos não classificados',
       'Relógios de pulso', 'Figuras de ação',
       'Controles remotos multimídia para consoles de videogames',
       'Figuras interativas para videogames',
       'Consoles de videogames, videogames e máquinas de fliperama',
       'Fontes de alimentação para consoles de videogames',
       'Skins para consoles e controles de videogames',
       'Relógios de parede', 'Cartões de memória',
       'Álbuns de música antigos', 'Monitores de computadores', 'Tênis',
       'Suportes para reprodutores de vídeo e consoles de videogames',
       'Câmeras e barras sensoras para consoles de videogames',
       'Capas e estojos para consoles de videogames',
       'Cabos e adaptadores de áudio e vídeo',
       'Óculos de realidade virtual', 'Veículos em miniatura', 'Livros',
       'Camisas de futebol', 'Copos e taças', 'Camisetas',
       'Álbuns de música', 'Brinqu

In [146]:
categorias = {
    'Videogames': '001',
    'Figuras de ação': '002',
    'Consoles de videogames': '003',
    'Gamepads e joysticks': '004',
    'Figuras interativas para videogames': '005',
    'Consoles de videogames, videogames e máquinas de fliperama': '006',
    'Relógios de pulso': '007',
    'Skins para consoles e controles de videogames': '008',
    'Camisetas': '009',
    'Livros': '010',
    'Veículos em miniatura': '011',
    'Camisas de futebol': '012',
    'Câmeras e barras sensoras para consoles de videogames': '013',
    'Controles remotos multimídia para consoles de videogames': '014',
    'Tênis': '015',
    'Brinquedos e hobbies': '016',
    'Produtos não classificados': '017',
    'Controles remotos para reprodutores de vídeo': '018',
    'Óculos de realidade virtual': '019',
    'Relógios de parede': '020',
    'Relógios de bolso': '021',
    'Luminárias de mesa e escritório': '022',
    'Canecas': '023',
    'Suportes para reprodutores de vídeo e consoles de videogames': '024',
    'Copos e taças': '025',
    'Álbuns de música antigos': '026',
    'Capas e estojos para consoles de videogames': '027',
    'Cartões de memória': '028',
    'Fontes de alimentação para consoles de videogames': '029',
    'Cabos e adaptadores de áudio e vídeo' : '030'
}
df['CATEGORY_ID'] = df['CATEGORY'].map(categorias)
df.columns

Index(['ITEM_ID', 'SKU', 'TITLE', 'QUANTITY', 'CHANNEL', 'MARKETPLACE_PRICE',
       'MSHOPS_PRICE', 'MSHOPS_PRICE_SYNC', 'CURRENCY_ID', 'DESCRIPTION',
       'SHIPPING_METHOD_MARKETPLACE', 'SHIPPING_METHOD_MSHOPS', 'LISTING_TYPE',
       'FEE_PER_SALE_MARKETPLACE', 'FEE_PER_SALE_MSHOPS', 'STATUS', 'CATEGORY',
       'CATEGORY_ID'],
      dtype='object')

In [148]:
def update_product_skus(data):
    
    current_year_month = datetime.now().strftime("%y%m")  # Ano e mês atual no formato "yyMM"
    
    # Preparando para rastrear informações adicionais sobre novos SKUs
    new_skus_list = []
    new_skus_details = []  # Lista para armazenar detalhes para novos SKUs incluindo ITEM_ID e TITLE

    # Certifique-se de que CATEGORY_ID tenha 3 dígitos
    data['CATEGORY_ID'] = data['CATEGORY_ID'].str.zfill(3)

    # Dicionário para rastrear o contador por categoria e mês
    category_counters = {}

    for category_id in data['CATEGORY_ID'].unique():
        category_mask = data['CATEGORY_ID'] == category_id

        # Filtra os SKUs existentes dessa categoria e mês
        existing_skus = data.loc[category_mask & data['SKU'].notna(), 'SKU']

        # Verifica o ano/mês atual para a categoria
        year_month = current_year_month

        # Extraímos os contadores do SKU, considerando a categoria e mês
        counters = existing_skus.str.extract(f"^{category_id}-{year_month}-(\\d{{4}})$")[0]
        valid_counters = pd.to_numeric(counters, errors='coerce').dropna().astype(int)
        
        # Define o próximo contador, baseado no maior contador existente, ou inicia em 1
        if not valid_counters.empty:
            next_counter = valid_counters.max() + 1
        else:
            next_counter = 1

        # Filtra os índices dos produtos sem SKU
        null_skus_indices = data.index[category_mask & data['SKU'].isna()]
        for idx in null_skus_indices:
            # Gera um SKU único com base na categoria, ano/mês e o contador
            new_sku = f"{category_id}-{year_month}-{next_counter:04d}"
            data.at[idx, 'SKU'] = new_sku
            new_skus_list.append(new_sku)
            new_skus_details.append({
                'TITLE': data.at[idx, 'TITLE'],
                'SKU': new_sku,
                'ITEM_ID': data.at[idx, 'ITEM_ID'],
            })

            # Incrementa o contador após gerar o SKU para o próximo
            next_counter += 1

    # Converte a lista de dicionários em DataFrame
    new_skus_data = pd.DataFrame(new_skus_details)  # Agora inclui ITEM_ID e TITLE

    return data, new_skus_data

def get_links(data):
    data['ITEM_LINK'] = data['ITEM_ID'].apply(
        lambda x: f"https://www.mercadolivre.com.br/anuncios/lista?filters=OMNI_ACTIVE|OMNI_INACTIVE|CHANNEL_NO_PROXIMITY_AND_NO_MP_MERCHANTS&page=1&search={x[3:]}" if pd.notnull(x) else "")

    data['URL'] = data.apply(
        lambda row: f"https://www.collectorsguardian.com.br/{row['ITEM_ID'][:3]}-{row['ITEM_ID'][3:]}-{row['TITLE'].replace(' ', '-').lower()}-_JM#item_id={row['ITEM_ID']}", 
        axis=1)

    return data
    
data, news = update_product_skus(df)
print(news)
data = get_links(data)
data.head(2)

                                                TITLE            SKU  \
0   Controle Ps1 - Hori Original Japao Cor Azul - ...  004-2501-0005   
1   Rain - Ps3 Exclusivo Sony - Jogo Incomum - Jap...  001-2501-0032   
2   Dead Or Alive 3 - Xbox Classic Japones - Lacrado   001-2501-0033   
3   Biohazard/ Resident Evil- Sega Saturn - Original   001-2501-0034   
4                         Rockman X2 - Super Famicom   001-2501-0035   
5       Final Fantasy X - Ps2 + Memory Card Original   001-2501-0036   
6   Mickey Mouse The Magical Quest Starring Para S...  001-2501-0037   
7                  Super Bomberman 2 - Super Famicom   001-2501-0038   
8   Jogo Super Mario Kart Para Super Famicom - Com...  001-2501-0039   
9   Turtles Tartarugas Cowabunga Collection Limite...  001-2501-0040   
10  Afrika Ps3 - Original Japonês - Exclusivo Play...  001-2501-0041   
11    Street Fighter Zero 2 - Sega Saturn - Original   001-2501-0042   
12                    Dragon Quest V - Super Famicom   001-2501-

,ITEM_ID,SKU,TITLE,QUANTITY,CHANNEL,MARKETPLACE_PRICE,MSHOPS_PRICE,MSHOPS_PRICE_SYNC,CURRENCY_ID,DESCRIPTION,SHIPPING_METHOD_MARKETPLACE,SHIPPING_METHOD_MSHOPS,LISTING_TYPE,FEE_PER_SALE_MARKETPLACE,FEE_PER_SALE_MSHOPS,STATUS,CATEGORY,CATEGORY_ID,ITEM_LINK,URL
0,MLB3420123395,021-2410-0001,Pocket Watch Limited Edition Super Mário Bros ...,1,Mercado Livre e Mercado Shops,400.0,350.0,No Vincular,R$,Apresentando o Relógio de Bolso Super Mario Br...,Mercado Envios grátis,Mercado Envios grátis,Clássico,NaN,NaN,Ativo,Relógios de bolso,021,https://www.mercadolivre.com.br/anuncios/lista...,https://www.collectorsguardian.com.br/MLB-3420...
1,MLB3420123395,021-2410-0005,Pocket Watch Limited Edition Super Mário Bros ...,1,Mercado Livre e Mercado Shops,400.0,350.0,No Vincular,R$,Apresentando o Relógio de Bolso Super Mario Br...,Mercado Envios grátis,Mercado Envios grátis,Clássico,NaN,NaN,Ativo,Relógios de bolso,021,https://www.mercadolivre.com.br/anuncios/lista...,https://www.collectorsguardian.com.br/MLB-3420...


In [154]:
data = data.drop_duplicates(subset='SKU')
data.shape

(793, 20)

In [160]:
def extract_first_src(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
    
        img_tag = soup.find('img', class_='ui-pdp-image ui-pdp-gallery__figure__image')
        if img_tag and 'src' in img_tag.attrs:
            return img_tag['src']
        return None  # Caso a tag ou atributo não seja encontrado
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return None

# Aplicando a função a cada link no DataFrame
data['IMG'] = data['URL'].apply(extract_first_src)

# Exibindo o DataFrame com a nova coluna
data.shape

Erro ao processar https://www.collectorsguardian.com.br/MLB-3771964457-metal-gear-rising-revengeance---collectors-edition-xbox-360-_JM#item_id=MLB3771964457: 429 Client Error: Too Many Requests for url: https://www.collectorsguardian.com.br/MLB-3771964457-metal-gear-rising-revengeance---collectors-edition-xbox-360-_JM#item_id=MLB3771964457
Erro ao processar https://www.collectorsguardian.com.br/MLB-4498386968-funko-pop-itachi-#578-anime-naruto-shippuden-special-edition-_JM#item_id=MLB4498386968: 404 Client Error: Not Found for url: https://www.collectorsguardian.com.br/MLB-4498386968-funko-pop-itachi-#578-anime-naruto-shippuden-special-edition-_JM%23item_id=MLB4498386968


(793, 21)

In [164]:
# Passo 1: Dividir o DataFrame em partes de 500 linhas
tamanho_maximo = 500
dataframes_divididos = np.array_split(data, np.ceil(len(data) / tamanho_maximo))

# Passo 2: Exportar cada parte para uma sheet diferente
with pd.ExcelWriter("WITHIMAGES.xlsx", engine='openpyxl') as writer:
    for i, df in enumerate(dataframes_divididos):
        sheet_name = f'PRODUTOS_PARTE_{i + 1}'  # Nome da sheet (ex: PRODUTOS_PARTE_1, PRODUTOS_PARTE_2, etc.)
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Arquivo Excel exportado com sucesso!")

C:\Users\nonak\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Arquivo Excel exportado com sucesso!
